In [1]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-4v1klns3
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-4v1klns3
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done


In [2]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-i633w0p0
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-i633w0p0
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [3]:
import pandas as pd

# 전처리된 CSV 로드
df = pd.read_csv("smishing_original.csv")

# 제거 대상 키워드 목록
keywords = [
    "택배", "카드 신청", "반품", "보관", "수거",
    "현관", "발급", "우편물", "우편함", "우편", r"(배송\s?예정|배달\s?예정)",
    "배송"
]

# 각 키워드에 대해 "국제|국외"가 포함되지 않은 경우만 제거
for keyword in keywords:
    condition = df["CN"].str.contains(keyword, na=False) & ~df["CN"].str.contains("국제|국외", na=False)
    df = df[~condition].copy()

#중복제거
df = df.drop_duplicates(subset=["CN"]).copy()

# 저장
df.to_csv("keyword_smishing.csv", index=False, encoding="utf-8-sig")
print("전처리 완료: keyword_smishing.csv 저장됨")

<ipython-input-3-ddfc5e1f600b>:15: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  condition = df["CN"].str.contains(keyword, na=False) & ~df["CN"].str.contains("국제|국외", na=False)


전처리 완료: keyword_smishing.csv 저장됨


In [4]:
# 데이터 받아오기

import pandas as pd
import re
from pykospacing import Spacing # 띄어쓰기 !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from hanspell import spell_checker
from tqdm import tqdm

tqdm.pandas()
spacing = Spacing()

df = pd.read_csv("keyword_smishing.csv")
spacing = Spacing()

# 맞춤법 교정 함수
def safe_spell_check(text):
    try:
        if isinstance(text, str) and 1 < len(text) <= 500:
            result = spell_checker.check(text)
            if hasattr(result, 'checked'):
                return result.checked
    except:
        pass
    return text

# 전처리 함수
def preprocess_text(text):

  ## 의미 없는 문자열 "ifg@" 제거
  text = text.replace("ifg@", "")

  ## url >> URL로 치환 | https나 http나 www로 시작하는 url 치환
  text = re.sub(r"(https?://|http?://|www\.)[^\s]+", "URL", text)

  ## 마스킹된 날짜를 DATE로 치환
  # yyyy년 MM월 dd일
  text = re.sub(r"\d{4}년\s?\d{1,2}월\s?\d{1,2}일", "DATE", text)

  # MM월 dd일
  text = re.sub(r"\d{1,2}월\s?\d{1,2}일", "DATE", text)

  # dd일만 나오는 경우 (예: "오늘 30일까지만 가능")
  text = re.sub(r"\d{1,2}일", "DATE", text)

  # yy년만 나오는 경우
  text = re.sub(r"\d{1,2}년", "DATE", text)

  # MM월만 나오는 경우
  text = re.sub(r"\d{1,2}월", "DATE", text)

  # yy년 MM월만 나오는 경우
  text = re.sub(r"\d{1,2}년\s?\d{1,2}월", "DATE", text)

  # MM/dd(일) 만 나오는 경우
  text = re.sub(r"\d{1,2}/\d{1,2}\(.\)", "DATE", text)

  # MM/dd(요일)
  text = re.sub(r"\d{1,2}/\d{1,2}\(\w+\)", "DATE", text)

  ## 마스킹된 시간을 TIME으로 치환
  # **:** 시:분 만 나오는 경우
  text = re.sub(r"\d{2}:\d{2}", "TIME", text)

  # *시 **시 *-*시간 *시간 *시~**시 **시-**시 **시 ~ **시 *시~**시 *시** *시-**시 **시~**시
  text = re.sub(r"\*{1,2}시", "TIME", text)
  text = re.sub(r"\*{1,2}시간", "TIME", text)
  text = re.sub(r"\*{1,2}시\s*[\-~]\s*\*{1,2}시", "TIME", text)

  # L*TT* >> LOTTO로 치환
  text = re.sub(r"L\*TT\*", "로또", text)


  ## 전화번호 및 *로 마스킹된 숫자를 전부 NUM으로 치환
  # 연속된 *, 숫자, -, _, 공백, 쉼표가 포함된 마스킹 형태 전부를 하나의 'NUM'으로 통일
  text = re.sub(r"[\*\d,\-_\s]{6,}", "NUM", text)

  # 한글,알파벳,공백을 제외한 나머지 문자 제거
  text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)


  # 띄어쓰기 보정
  text = spacing(text)

  # 맞춤법 보정
  text = safe_spell_check(text)

  return text


df["clean_df"] = df["CN"].progress_apply(preprocess_text)

df.to_csv("smishing_clean.csv", index=False, encoding="utf-8-sig")

100%|██████████| 12210/12210 [55:36<00:00,  3.66it/s]
